In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from pyCascade import probePost, physics, utils, probeReadWrite
from pyCascade.probeReadWrite import read_probes_file_switch
from filloutVentilationStats import *
from matplotlib import pyplot as plt
from matplotlib import cm, colors
import matplotlib.ticker as ticker
import numpy as np
import scipy as sp
import os
from IPython.core.debugger import set_trace
import pandas as pd
import seaborn as sns
from cycler import cycler
import plotly.express as px
import plotly
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import statsmodels.api as sm
import warnings

plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')

## Runs

In [ ]:
multiRun_dir = f"{home_dir}/CHARLES/multiRuns/"
plotFolder = f"{multiRun_dir}"

## Read in results

In [ ]:
flowStatsMI = pd.read_csv(f"{multiRun_dir}/flowStatsMI.csv", index_col = [0,1])
roomVentilationMI = pd.read_csv(f"{multiRun_dir}/roomVentilationMI.csv", index_col = [0,1])

flowStatsMI = replace_sl_with_h_df(flowStatsMI)
roomVentilationMI = replace_sl_with_h_df(roomVentilationMI)

flowStatsMI.loc[flowStatsMI["houseType"] == "-1-0", "slAll"] = True
roomVentilationMI.loc[roomVentilationMI["houseType"] == "-1-0", "slAll"] = True

# flowStatsMI = flowStatsMI[flowStatsMI['slAll'] == True]
# roomVentilationMI = roomVentilationMI[roomVentilationMI['slAll'] == True]

In [ ]:
# runSubset = [
#     2460, 2461, 2462, 
#     2470, 2471, 2472, 
#     2480, 2481, 2482, 
#     2490, 2491, 2492, 
#     3160, 3161, 3162, 
#     3170, 3171, 3172, 
#     3180, 3181, 3182, 
#     3190, 3191, 3192, 
# ]

# runSubset = [
#     2530, 2531, 2532, 
#     # 2540, 2541, 2542, 
#     # 2550, 2551, 2552, 
#     # 2560, 2561, 2562, 
#     # 3210, 3211, 3212, 
#     # 3220, 3221, 3222, 
#     # 3230, 3231, 3232, 
#     # 3240, 3241, 3242, 
# ]

# flowStatsMI = flowStatsMI.loc[runSubset]

# roomVentilationMI = roomVentilationMI.loc[runSubset]

In [ ]:
flowStatsMI = combine_stats(flowStatsMI, ["WS", "delT", "SS", "C", "A", "slAll"], index_col = "csId")
roomVentilationMI = combine_stats(roomVentilationMI,   ["WS", "delT", "SS", "C", "A", "slAll"], index_col = "csId")

In [ ]:
foo = roomVentilationMI["q-D-room"] / roomVentilationMI["q-D-room-Norm"]
# histogram
fig, ax = plt.subplots()
ax.hist(foo, bins=np.arange(0.001, 21.001), color='c', edgecolor='black', linewidth=1.2)

# Violins


## Wall Convection Violins

In [ ]:
value_vars = ["mean-mass_flux(S)-roomCeil-perA-Norm", "mean-mass_flux(S)-roomWalls-perA-Norm", "mean-mass_flux(S)-roomFloor-perA-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.5, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()


value_vars = ["mean-mass_flux(S)-roomCeil-h-Norm", "mean-mass_flux(S)-roomWalls-h-Norm", "mean-mass_flux(S)-roomFloor-h-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.5, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()


## Volume Violins

In [ ]:
rooms = ["room5", "room4", "room3", "room2", "room1", "room0"]
value_vars = [f"mean-S-{r}" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
# plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"mean-S-room"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)
# plotdf["value"]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()


### Temperature-Tracer Comparison/Check

In [ ]:
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == False]

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.scatterplot, "c-T-room", "mean-S-room", "delT", hue_order = [0, 5])
g.add_legend()


## Tracer Decay Violins

In [ ]:

value_vars = [f"tau-D-{r}" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"tau-D-room"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"tau-D-room"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

In [ ]:

value_vars = [f"q-D-{r}-Norm" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"q-D-room-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"q-D-room-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

## Decay Heat Flux Violins

In [ ]:

value_vars = [f"q-D-{r}-Norm" for r in rooms]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

value_vars = [f"q-D-room-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "delT", order = value_vars, hue_order = [0, 5], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

## Mass Flux Violins

In [ ]:
value_vars = [f"mean-mass_flux-Norm"]

plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["delT"] == 5]
id_vars = [ c for c in plotdf.columns.values if '-' not in c]
plotdf = pd.melt(plotdf, id_vars=id_vars, value_vars = value_vars)

g = sns.FacetGrid(plotdf, col = "roomType", col_wrap = 2, col_order = ["corner", "single", "dual", "cross"], aspect = 1.25, legend_out = True)
g.map(sns.violinplot, "value", "variable", "SS", order = value_vars, hue_order = [True, False], fill = False, inner = "quart", split = True, cut = 0)
g.add_legend()

In [ ]:
# Create a figure with 2x5 subplot grid
fig, axs = plt.subplots(2, 5, figsize=(24, 12), dpi=140)
axs = axs.flatten()  # Flatten for easier indexing
Ri_values = sorted(flowStatsMI["Ri"].unique())

# Define combinations of Ri, SS, and slAll
combinations = [
    (Ri_values[2], False, True), 
    (Ri_values[1], False, True), 
    (Ri_values[2], True,  True), 
    (Ri_values[1], True,  True), 
    (Ri_values[0], True,  True), 
    (Ri_values[2], False, False),
    (Ri_values[1], False, False),
    (Ri_values[2], True,  False),
    (Ri_values[1], True,  False),
    (Ri_values[0], True,  False),
]

# Define QOIs (Quantities of Interest)
qois = ["mean-mass_flux(S)-Norm", "mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "q-D-room-Norm"]
new_labels = ["$\\bar{Q}_{s}$", "$\\bar{Q}$", "$\overline{|Q|}$", "$\\bar{Q}_d$"]

# Value to normalize by
rho = 1.0  # Using 1.0 as seen in the code below

# Create plots for each combination
for i, (ri_val, ss_val, sl_val) in enumerate(combinations):
    # Filter data for this combination
    plotdf = roomVentilationMI.copy()
    plotdf = plotdf[np.isclose(plotdf["Ri"], ri_val)]
    plotdf = plotdf[plotdf["SS"] == ss_val]
    plotdf = plotdf[plotdf["slAll"] == sl_val]
    
    # Normalize values
    plotdf["mean-mass_flux(S)-Norm"] /= -rho
    plotdf["mean-mass_flux-Norm"] /= rho
    plotdf["q-D-room-Norm"] /= rho
    
    # Group by room type
    x_var = "roomType"
    plotdfMelted = plotdf.melt(id_vars=[x_var], value_vars=qois, var_name="QOI", value_name="Value")
    
    # Create the box plot for this subplot
    sns.boxplot(data=plotdfMelted, x=x_var, y="Value", hue="QOI", palette="Set2", ax=axs[i])
    
    # Customize the subplot
    axs[i].set_title(f"Ri={ri_val:.4f}, {'Steady State' if ss_val else 'Non-Steady State'}, {'With Skylights' if sl_val else 'No Skylights'}", fontsize=14)
    axs[i].set_xlabel("Room Type" if i >= 5 else "", fontsize=12)
    axs[i].set_ylabel("Normalized Ventilation Rate" if i % 5 == 0 else "", fontsize=12)
    
    # Set legend with custom labels
    if i == 0:  # Only add detailed legend to first subplot
        handles, labels = axs[i].get_legend_handles_labels()
        axs[i].legend(handles, new_labels, title="QOI", loc='upper right')
    else:
        axs[i].get_legend().remove()  # Remove redundant legends

# Add overall title
fig.suptitle("Ventilation Metrics by Room Type, Richardson Number, Steady State, and Skylights", fontsize=16, y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Make room for suptitle

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 10), dpi=140)

plotdf = roomVentilationMI.copy()
plotdf = plotdf[plotdf["SS"] == True]
plotdf["mean-mass_flux(S)-Norm"] /= -rho
plotdf["mean-mass_flux-Norm"] /= rho
plotdf["q-D-room-Norm"] /= rho

qois = ["mean-mass_flux(S)-Norm", "mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "q-D-room-Norm"]

group = ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"]

for i in range(2):
    subplot_df = plotdf[(plotdf["houseType"] == "sl") == i]
    plotdfGrouped = subplot_df.groupby(group, as_index=False).mean(numeric_only=True)
    
    # Create a new column to identify each QOI
    plotdfMelted = plotdfGrouped.melt(id_vars=["roomType"], value_vars=qois, var_name="QOI", value_name="Value")
    
    # Create the box plot with QOI as hue
    sns.boxplot(data=plotdfMelted, x="roomType", y="Value", hue="QOI", palette="Set2", ax=axs[i])
    
    # Customize the plot
    axs[i].legend(title="QOI", loc='upper right')
    # map legend to new labels
    new_labels = ["$\\bar{Q}_{s}$", "$\\bar{Q}$", "$\overline{|Q|}$", "$\\bar{Q}_d$"]
    for t, l in zip(axs[i].get_legend().texts, new_labels): t.set_text(l)
    axs[i].tick_params(axis='both', which='major', labelsize=12)
    axs[i].set_xlabel("Room Type" if i == 1 else "", fontsize=12)
    axs[i].set_ylabel("Normalized Ventilation Rate", fontsize=12)
    axs[i].set_title(f"Skylights: {bool(i)}", fontsize=14)

# Add a main title
fig.suptitle("Box Plot for Different QOIs by Temperature Difference", fontsize=16, y=0.98)
plt.tight_layout()

In [ ]:
plotdf_all = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf_all = plotdf_all[plotdf_all["SS"] == True]
plotdf_all["mean-mass_flux(S)-Norm"] /= -rho
plotdf_all["mean-mass_flux-Norm"] /= rho
plotdf_all["q-D-room-Norm"] /= rho

r_lim = [.59, .59, .59, .059]
# r_lim = [r/2 for r in r_lim]
fig, axs = plt.subplots(1, 4, subplot_kw={'projection': 'polar'}, figsize = (16, 5), dpi = 600, layout="tight")
colors = plt.cm.get_cmap("Set2").colors  # Use Matplotlib's Set2 colormap

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    # for j, delT in enumerate([0, 5]):
    plotdf = plotdf_all.copy()
    plotdf = plotdf_all[plotdf_all["roomType"] == room]
    # plotdf = plotdf[plotdf["delT"] == delT]
    
    x_var = "AofA"
    y_vars = ["mean-mass_flux(S)-Norm", "mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "q-D-room-Norm"]

    # Create the polar plot
    ax = axs[i]
    ax.set_ylim(0, r_lim[i])
    
    # Customize gridlines to be lighter
    ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
    # Set the radial labels to use scientific notation
    ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.set_rlabel_position(0)

    # Create the bars
    plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True) #CHANGED TO MEDIAN
    angles = np.deg2rad(plotdfGrouped[x_var])
    radii = np.mean(plotdfGrouped[y_vars].values, axis = 1)
    ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth = 0.25) #CHANGED TO MEDIAN
    for k, y_var in enumerate(y_vars):
        width = np.pi/35 * 5 / 4
        offset = (len(y_vars)/2.5 - k)*width
        angles = np.deg2rad(plotdfGrouped[x_var]) + offset
        radii = plotdfGrouped[y_var].values
        # color = "lightgrey"
        color = colors[k]
        ax.bar(angles, radii, width=width, color=color, edgecolor = 'white', linewidth = .5, alpha = 1)
        
    # Adding a legend for the custom error bars
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

    # Customize plot
    ax.set_theta_direction(1)  # counter clockwise
    ax.set_theta_offset(np.pi)  # Starting from the right
    ax.tick_params(axis='both', which='major', labelsize=14)
    ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
    yticks = ax.get_yticks()
    ax.set_yticks(yticks[1::2])
    ax.title.set_text(room)

# plt.tight_layout()

In [ ]:
plotdf_all = roomVentilationMI.copy()
plotdf_all = plotdf_all[plotdf_all["SS"] == True]
plotdf_all["mean-mass_flux(S)-Norm"] /= -rho
plotdf_all["mean-mass_flux-Norm"] /= rho
plotdf_all["q-D-room-Norm"] /= rho

r_lim = [.59, .59, .59, .19]

fig, axs = plt.subplots(2, 4, subplot_kw={'projection': 'polar'}, figsize = (16, 10), dpi = 600, layout="tight")
colors = plt.cm.get_cmap("Set2").colors  # Use Matplotlib's Set2 colormap

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j in range(2):
        plotdf = plotdf_all.copy()
        plotdf = plotdf_all[plotdf_all["roomType"] == room]
        plotdf = plotdf[(plotdf["houseType"] == "sl") == j]
    
        x_var = "AofA"
        y_vars = ["mean-mass_flux(S)-Norm", "mean-mass_flux-Norm", "mean-sn_prod(abs(u))-Norm", "q-D-room-Norm"]

        # Create the polar plot
        ax = axs[j, i]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars
        plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True) #CHANGED TO MEDIAN
        angles = np.deg2rad(plotdfGrouped[x_var])
        radii = np.mean(plotdfGrouped[y_vars].values, axis = 1)
        ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth = 0.25) #CHANGED TO MEDIAN
        for k, y_var in enumerate(y_vars):
            width = np.pi/35 * 5 / 3
            offset = (len(y_vars)/2.5 - k)*width
            angles = np.deg2rad(plotdfGrouped[x_var]) + offset
            radii = plotdfGrouped[y_var].values / rho
            # color = "lightgrey"
            color = colors[k]
            ax.bar(angles, radii, width=width, color=color, edgecolor = 'white', linewidth = .5, alpha = 1)
        
        # Adding a legend for the custom error bars
        handles, labels = ax.get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        # ax.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1))

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])
        if j == 0:
            ax.title.set_text(room)
        if i == 0:
            ax.set_ylabel(f"Skylights: {bool(j)}", fontsize=16)

In [ ]:
roomVentilationMI["Ri"]

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 12), dpi=140)
axs = axs.flatten()  # Flatten array for easier indexing

plotdf = roomVentilationMI.copy()
plotdf = plotdf[plotdf["SS"] == True]

qois = ["mean-mass_flux(S)-roomCeil-perA-Norm", "mean-mass_flux(S)-roomWalls-perA-Norm", "mean-mass_flux(S)-roomFloor-perA-Norm"]

group = ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"]

plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)

x_var = "roomType"

# Get unique Ri values and sort them
unique_ri = sorted(plotdfGrouped["Ri"].unique())

# Create subplots for each combination
subplot_idx = 0
for i, has_skylight in enumerate([False, True]):
    for j, ri_value in enumerate(unique_ri):
        if subplot_idx >= len(axs):
            break
            
        # Filter data for current subplot
        current_df = plotdfGrouped.copy()
        if has_skylight:
            current_df = current_df[current_df["houseType"] == "sl"]
        else:
            current_df = current_df[current_df["houseType"] != "sl"]
            
        current_df = current_df[np.isclose(current_df["Ri"], ri_value)]
        
        # Create melted dataframe for this subplot
        plotdfMelted = current_df.melt(id_vars=[x_var], value_vars=qois, var_name="QOI", value_name="Value")

        # Create the box plot with QOI as hue
        sns.boxplot(data=plotdfMelted, x=x_var, y="Value", hue="QOI", palette="Set1", ax=axs[subplot_idx])
        
        # Add subplot title
        axs[subplot_idx].set_title(f"Ri = {ri_value:.3f}" + (", With Skylights" if has_skylight else ", No Skylights"), fontsize=14)
        
        # Customize each subplot
        axs[subplot_idx].tick_params(axis='both', which='major', labelsize=12)
        axs[subplot_idx].set_xlabel("Room Type" if i == 1 else "", fontsize=12)
        axs[subplot_idx].set_ylabel("Flux per Area" if j == 0 else "", fontsize=12)
        
        # Rename the legend labels
        handles, labels = axs[subplot_idx].get_legend_handles_labels()
        new_labels = ["Ceiling", "Walls", "Floor"]
        
        # Only show legend on the right plots
        if j == len(unique_ri) - 1:
            axs[subplot_idx].legend(handles, new_labels, title="Surface", loc='upper right')
        else:
            axs[subplot_idx].get_legend().remove()
            
        subplot_idx += 1

# Hide any unused subplots
for i in range(subplot_idx, len(axs)):
    axs[i].set_visible(False)

# Add overall title
fig.suptitle("Wall Convection Fluxes by Room Surface for Different Richardson Numbers", fontsize=16, y=0.98)
plt.tight_layout()


In [ ]:
plotdf_all = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf_all = plotdf_all[plotdf_all["SS"] == True]

# Get unique Richardson numbers
unique_ri = sorted(plotdf_all["Ri"].unique())

r_lim = [2.5, 2.5, 2.5, .5]
r_lim = [r/1000 for r in r_lim]
fig, axs = plt.subplots(len(unique_ri), 4, subplot_kw={'projection': 'polar'}, figsize=(16, 12), dpi=600, layout="tight")
colors = plt.cm.get_cmap("Set1").colors  # Use Matplotlib's Set1 colormap

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j, ri_value in enumerate(unique_ri):
        plotdf = plotdf_all.copy()
        plotdf = plotdf_all[plotdf_all["roomType"] == room]
        plotdf = plotdf[np.isclose(plotdf["Ri"], ri_value)]
    
        x_var = "AofA"
        y_vars = ["mean-mass_flux(S)-roomCeil-perA-Norm", "mean-mass_flux(S)-roomWalls-perA-Norm", "mean-mass_flux(S)-roomFloor-perA-Norm"]

        # Create the polar plot
        ax = axs[j, i]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars only if there's data
        if not plotdf.empty:
            plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True)
            angles = np.deg2rad(plotdfGrouped[x_var])
            radii = np.mean(plotdfGrouped[y_vars].values, axis=1)
            ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth=0.25)
            
            for k, y_var in enumerate(y_vars):
                width = np.pi/35 * 5 / 3
                offset = (len(y_vars)/2.5 - k)*width
                angles = np.deg2rad(plotdfGrouped[x_var]) + offset
                radii = plotdfGrouped[y_var].values / rho
                color = colors[k]
                ax.bar(angles, radii, width=width, color=color, edgecolor='white', linewidth=.5, alpha=1)

        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])
        
        if j == 0:
            ax.title.set_text(room)
        if i == 0:
            ax.set_ylabel(f"Ri = {ri_value:.4f}", fontsize=16)

# Add a legend at the bottom of the figure
legend_labels = ["Ceiling", "Walls", "Floor"]
legend_handles = [plt.Rectangle((0,0), 1, 1, color=colors[i]) for i in range(len(y_vars))]
fig.legend(legend_handles, legend_labels, loc='lower center', ncol=3, fontsize=14, bbox_to_anchor=(0.5, 0.02))

plt.suptitle("Wall Convection Flux per Area by Richardson Number", fontsize=18, y=0.98)

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 12), dpi=140)

plotdf = roomVentilationMI.copy()
plotdf = plotdf[plotdf["SS"] == True]

qois = ["mean-mass_flux(S)-roomCeil-h-Norm", "mean-mass_flux(S)-roomWalls-h-Norm", "mean-mass_flux(S)-roomFloor-h-Norm"]

group = ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"]

plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)

# Get unique Ri values
unique_ri = sorted(plotdfGrouped["Ri"].unique())

# Create 2x3 subplots (2 rows for skylight/no skylight, 3 columns for Richardson numbers)
for i, has_skylight in enumerate([False, True]):
    for j, ri_value in enumerate(unique_ri):
        # Filter data for current subplot
        current_df = plotdfGrouped.copy()
        if has_skylight:
            current_df = current_df[current_df["houseType"] == "sl"]
        else:
            current_df = current_df[current_df["houseType"] != "sl"]
            
        current_df = current_df[np.isclose(current_df["Ri"], ri_value)]
        
        # Create melted dataframe for this subplot
        plotdfMelted = current_df.melt(id_vars=["roomType"], value_vars=qois, var_name="QOI", value_name="Value")

        # Create the box plot with QOI as hue
        sns.boxplot(data=plotdfMelted, x="roomType", y="Value", hue="QOI", palette="Set1", ax=axs[i, j])
        
        # Add subplot title
        axs[i, j].set_title(f"Ri = {ri_value:.3f}", fontsize=14)
        
        # Customize each subplot
        axs[i, j].tick_params(axis='both', which='major', labelsize=12)
        axs[i, j].set_xlabel("Room Type" if i == 1 else "", fontsize=12)
        axs[i, j].set_ylabel("Heat Transfer Coefficient" if j == 0 else "", fontsize=12)
        
        # Rename the legend labels
        handles, labels = axs[i, j].get_legend_handles_labels()
        new_labels = ["Ceiling", "Walls", "Floor"]
        
        # Only show legend on the right plots
        if j == len(unique_ri) - 1:
            axs[i, j].legend(handles, new_labels, title="Surface", loc='upper right')
        else:
            axs[i, j].get_legend().remove()

# Add row labels for skylight information
fig.text(0.01, 0.75, "No Skylights", fontsize=16, rotation=90)
fig.text(0.01, 0.25, "With Skylights", fontsize=16, rotation=90)

# Add overall title
fig.suptitle("Heat Transfer Coefficients by Room Surface for Different Richardson Numbers", fontsize=16, y=0.98)
plt.tight_layout(rect=[0.03, 0, 1, 0.95])  # Adjust layout to make room for row labels


In [ ]:
plotdf_all = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf_all = plotdf_all[plotdf_all["SS"] == True]

# Get unique Richardson numbers
unique_ri = sorted(plotdf_all["Ri"].unique())

r_lim = [8.9, 8.9, 8.9, 8.9]
# r_lim = [r/2 for r in r_lim]
fig, axs = plt.subplots(len(unique_ri), 4, subplot_kw={'projection': 'polar'}, figsize=(16, 12), dpi=600, layout="tight")
colors = plt.cm.get_cmap("Set1").colors  # Use Matplotlib's Set1 colormap

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j, ri_value in enumerate(unique_ri):
        plotdf = plotdf_all.copy()
        plotdf = plotdf_all[plotdf_all["roomType"] == room]
        plotdf = plotdf[np.isclose(plotdf["Ri"], ri_value)]
    
        x_var = "AofA"
        y_vars = ["mean-mass_flux(S)-roomCeil-h-Norm", "mean-mass_flux(S)-roomWalls-h-Norm", "mean-mass_flux(S)-roomFloor-h-Norm"]

        # Create the polar plot
        ax = axs[j, i]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars only if there's data
        if not plotdf.empty:
            plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True)
            angles = np.deg2rad(plotdfGrouped[x_var])
            radii = np.mean(plotdfGrouped[y_vars].values, axis=1)
            ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth=0.25)
            
            for k, y_var in enumerate(y_vars):
                width = np.pi/35 * 5 / 3
                offset = (len(y_vars)/2.5 - k)*width
                angles = np.deg2rad(plotdfGrouped[x_var]) + offset
                radii = plotdfGrouped[y_var].values
                color = colors[k]
                ax.bar(angles, radii, width=width, color=color, edgecolor='white', linewidth=.5, alpha=1)
        
        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])
        
        if j == 0:
            ax.title.set_text(room)
        if i == 0:
            ax.set_ylabel(f"Ri = {ri_value:.4f}", fontsize=16)
 
# Add a legend at the bottom of the figure
legend_labels = ["Ceiling", "Walls", "Floor"]
legend_handles = [plt.Rectangle((0,0), 1, 1, color=colors[i]) for i in range(len(y_vars))]
fig.legend(legend_handles, legend_labels, loc='lower center', ncol=3, fontsize=14, bbox_to_anchor=(0.5, 0.02))

plt.suptitle("Heat Transfer Coefficients by Surface and Richardson Number", fontsize=18, y=0.98)

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(18, 12), dpi=600, layout="tight")

plotdf = roomVentilationMI.copy()
plotdf = plotdf[plotdf["SS"] == True]

# Define layers
qois = [f"q-D-room{k}-Norm" for k in range(6)]

group = ["roomType", "WS", "Ri", "delT", "houseType", "C", "AofA"]

plotdfGrouped = plotdf.groupby(group, as_index=False).mean(numeric_only=True)

# Get unique Ri values
unique_ri = sorted(plotdf["Ri"].unique())

# Create subplot titles
subplot_titles = [["No Skylights, Ri = {:.3f}", "No Skylights, Ri = {:.3f}", "No Skylights, Ri = {:.3f}"], 
                  ["With Skylights, Ri = {:.3f}", "With Skylights, Ri = {:.3f}", "With Skylights, Ri = {:.3f}"]]

for i, has_skylight in enumerate([False, True]):
    for j, ri_value in enumerate(unique_ri):
        # Filter data for current subplot
        current_df = plotdfGrouped.copy()
        if has_skylight:
            current_df = current_df[current_df["houseType"] == "sl"]
        else:
            current_df = current_df[current_df["houseType"] != "sl"]
            
        current_df = current_df[np.isclose(current_df["Ri"], ri_value)]
        
        # Create melted dataframe for this subplot
        plotdfMelted = current_df.melt(id_vars=["roomType"], value_vars=qois, var_name="QOI", value_name="Value")

        # Create the box plot with QOI as hue
        sns.boxplot(data=plotdfMelted, x="roomType", y="Value", hue="QOI", palette="Accent", ax=axs[i, j])
        
        # Add subplot title
        axs[i, j].set_title(subplot_titles[i][j].format(ri_value), fontsize=14)
        
        # Customize each subplot
        axs[i, j].tick_params(axis='both', which='major', labelsize=12)
        axs[i, j].set_xlabel("Room Type" if i == 1 else "", fontsize=12)
        axs[i, j].set_ylabel("Normalized Ventilation Rate" if j == 0 else "", fontsize=12)
        
        # Only show legend on the right plots
        if j == 2:
            handles, labels = axs[i, j].get_legend_handles_labels()
            new_labels = [f"Layer {k}" for k in range(6)]
            axs[i, j].legend(handles, new_labels, title="Room Layers", loc='upper right')
        else:
            axs[i, j].get_legend().remove()

# Add overall title
fig.suptitle("Ventilation Rates by Room Layer for Different Richardson Numbers", fontsize=16, y=0.98)
plt.tight_layout()

In [ ]:
n_layers = 6

r_lim = [.59, .59, .59, .059]
r_lim = [r/n_layers for r in r_lim]
# Get unique Richardson numbers
unique_ri = sorted(roomVentilationMI["Ri"].unique())

fig, axs = plt.subplots(len(unique_ri), 4, subplot_kw={'projection': 'polar'}, figsize=(16, 12), dpi=600, layout="tight")
colors = plt.cm.get_cmap("Accent").colors  # Use Matplotlib's Accent colormap

# Helper function to convert 'rgb(r, g, b)' to (r/255, g/255, b/255) tuple
def rgb_to_mpl(rgb_str):
    rgb = rgb_str.replace('rgb(', '').replace(')', '').split(',')
    return tuple(int(val) / 255 for val in rgb)

for i, room in enumerate(["corner", "cross", "dual", "single"]):
    for j, ri_value in enumerate(unique_ri):
        plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
        plotdf = plotdf[plotdf["SS"] == True]
        plotdf = plotdf[plotdf["roomType"] == room]
        plotdf = plotdf[np.isclose(plotdf["Ri"], ri_value)]
    
        x_var = "AofA"
        y_var = "q-D-room-Norm" #"mean-sn_prod(abs(u))-Norm"

        # Create the polar plot
        ax = axs[j, i]
        ax.set_ylim(0, r_lim[i])
    
        # Customize gridlines to be lighter
        ax.grid(color='lightgrey', linestyle='--', linewidth=1, alpha=0.5)
        # Set the radial labels to use scientific notation
        ax.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax.set_rlabel_position(0)

        # Create the bars only if there's data
        if not plotdf.empty:
            plotdfGrouped = plotdf.groupby(x_var, as_index=False).median(numeric_only=True)
            angles = np.deg2rad(plotdfGrouped[x_var])
            radii = plotdfGrouped[y_var].values / n_layers
            ax.bar(angles, radii, width=np.pi/6, color='white', edgecolor='black', linewidth=0.25)
            
            for k in range(n_layers):
                width = np.pi/35
                offset = (n_layers/2.5 - k)*width
                angles = np.deg2rad(plotdfGrouped[x_var]) + offset
                radii = plotdfGrouped[f"q-D-room{k}-Norm"].values
                color = colors[k]
                ax.bar(angles, radii, width=width, color=color, edgecolor='white', linewidth=.5, alpha=1)
        
        # Customize plot
        ax.set_theta_direction(1)  # counter clockwise
        ax.set_theta_offset(np.pi)  # Starting from the right
        ax.tick_params(axis='both', which='major', labelsize=14)
        ax.yaxis.get_offset_text().set_fontsize(16)  # X-axis scientific notation font size
        yticks = ax.get_yticks()
        ax.set_yticks(yticks[1::2])
        
        if j == 0:
            ax.title.set_text(room)
        if i == 0:
            ax.set_ylabel(f"Ri = {ri_value:.4f}", fontsize=16)

# Add a legend at the bottom of the figure
legend_labels = [f"Layer {k}" for k in range(n_layers)]
legend_handles = [plt.Rectangle((0,0), 1, 1, color=colors[i]) for i in range(n_layers)]
fig.legend(legend_handles, legend_labels, loc='lower center', ncol=6, fontsize=14, bbox_to_anchor=(0.5, 0.02))

plt.suptitle("Room Layer Ventilation Rates by Richardson Number", fontsize=18, y=0.98)

In [ ]:
# Create separate figures for different Richardson number values
plotdf = roomVentilationMI[roomVentilationMI["houseType"] != "sl"]
plotdf = plotdf[plotdf["SS"] == True]

# Define colors and markers for surfaces
colors = {'Ceil': 'blue', 'Walls': 'green', 'Floor': 'red'}

# Get unique Richardson numbers and sort them
unique_ri = sorted(plotdf["Ri"].unique())

# Process each Richardson number in a separate figure
for ri_value in unique_ri:
    # Filter data for current Richardson number
    ri_df = plotdf[np.isclose(plotdf["Ri"], ri_value)]
    
    # Create a new figure
    fig, axs = plt.subplots(2, 2, figsize=(17, 10))
    axs = axs.flatten()
    
    # Create custom legend handles for surface types
    surface_handles = [plt.Line2D([0], [0], marker='o', color='w', 
                               markerfacecolor=color, markersize=10) 
                    for color in colors.values()]
    surface_labels = list(colors.keys())
    
    # Create scatter plots for each room type
    for i, room_type in enumerate(["corner", "single", "dual", "cross"]):
        room_data = ri_df[ri_df["roomType"] == room_type]
        line_handles = []
        line_labels = []
        
        # Plot for each surface type (walls, ceiling, floor)
        for surface, y_var in zip(['Ceil', 'Walls', 'Floor'], 
                               ['mean-mass_flux(S)-roomCeil-h-Norm', 
                                'mean-mass_flux(S)-roomWalls-h-Norm', 
                                'mean-mass_flux(S)-roomFloor-h-Norm']):
            
            # Create scatter plot using seaborn
            sns.scatterplot(
                x="mean-mass_flux-Norm", 
                y=y_var,
                style="delT",
                data=room_data, 
                color=colors[surface],
                alpha=0.7,
                ax=axs[i],
                legend="brief"
            )
            
            # Add best fit line for each surface type
            if len(room_data) > 1:  # Ensure we have enough data for fitting
                x_data = room_data["mean-mass_flux-Norm"].values
                y_data = room_data[y_var].values
                
                # Only attempt fit if we have valid data
                if len(x_data) > 0 and not np.isnan(x_data).any() and not np.isnan(y_data).any():
                    m, b = np.polyfit(x_data, y_data, 1)
                    corr = np.corrcoef(x_data, y_data)[0, 1]
                    
                    # Create clean x range for line plot
                    x_line = np.linspace(min(x_data), max(x_data), 100)
                    
                    line = axs[i].plot(x_line, m*x_line + b, '-', color=colors[surface], 
                              alpha=0.8, linewidth=1.5)[0]
                    
                    # Save handle and label for the first subplot only
                    line_handles.append(line)
                    line_labels.append(f"{surface} fit (m={m:.2f}, b={b:.2f}, r={corr:.2f})")
        
        # Customize each subplot
        axs[i].set_title(f"{room_type.capitalize()} Room", fontsize=14)
        axs[i].set_xlabel("Normalized Mean Mass Flow" if i > 1 else "", fontsize=12)
        axs[i].set_ylabel("Convection Coefficient" if i % 2 == 0 else "", fontsize=12)
        axs[i].grid(True, linestyle='--', alpha=0.5)

        # Add mini-legend for fit lines only in each subplot
        axs[i].legend(handles=line_handles, 
                     labels=line_labels, 
                     loc='upper right', fontsize=9, ncol=1)

    fig.suptitle(f"Surface Convection Coefficients vs Mean Mass Flow (Ri = {ri_value:.4f})", fontsize=16, y=0.98)
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.15)  # Make room for the legends
